## Problem: To predict if certain customer will claim warranty for a given product

- Data fields
- Region - Customer region details
- state - Current location of customer
- Area - Area_Urban/rural
- City- Customers current located city
- Consumer_profile- Customer's work profile
- Product_category- Product category
- Product_type- Type of the product_Tv/Ac
- AC_1001_Issue- 1001 is failure of Compressor in AC
- AC_1002_Issue- 1002 is failure of Condenser Coil in AC
- AC_1003_Issue- 1003 is failure of Evaporator Coil in AC
- TV_2001_Issue- 2001 is failure of power supply in Tv
- TV_2002_Issue- 2002 is failure of Inverter in Tv
- TV_2003_Issue- 2003 is failure of Motherboard in Tv
- claim_value- Customer's claim amount in Rs
- Service_Centre- 7 Different service centers
- Product_Age- Duration of the product purchased by customer
- Purchased_from- From where product is purchased
- Call_details- call duration in mins
- Purpose- Purpose_compliant-Compliant raised by customer claim- claimed for the product Other- Other categories out of this
- Fraud- '1'- fradulent claim, '0' Genuin claim

NOTE: '0' means to replace the component, '1' means partial damage of the component and with servicing component good work and '2' no issue with the component. This is for all the columns coded with '0','1','2'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.svm import SVC # For SVM model
from sklearn.tree import DecisionTreeClassifier # For Decission Tree Classifier
from sklearn import tree
from ipywidgets import interact,interactive
from sklearn.ensemble import RandomForestClassifier # For random Forest Classifier
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression # For Logistic Regression
from xgboost import XGBClassifier,plot_importance # For XGBoost Classifier
from catboost import CatBoostClassifier # For Cat Boost Classifier
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [2]:
data = pd.read_csv("data/df_Clean.csv").iloc[:,1:]
data.head()

,Region,State,Area,City,Consumer_profile,Product_category,Product_type,AC_1001_Issue,AC_1002_Issue,AC_1003_Issue,TV_2001_Issue,TV_2002_Issue,TV_2003_Issue,Claim_Value,Service_Centre,Product_Age,Purchased_from,Call_details,Purpose,Fraud
0,South,Karnataka,Urban,Bangalore,Business,Entertainment,TV,0,0,0,1,2,0,15000.0,10,60,Manufacturer,0.5,Complaint,1
1,South,Karnataka,Rural,Bangalore,Business,Household,AC,1,1,0,0,0,0,20000.0,12,10,Dealer,1.0,Complaint,0
2,North,Haryana,Urban,Chandigarh,Personal,Household,AC,0,1,2,0,0,0,18000.0,14,10,Dealer,1.4,Claim,0
3,South,Tamil Nadu,Urban,Chennai,Business,Entertainment,TV,0,0,0,1,1,0,12000.0,16,20,Manufacturer,2.0,Complaint,0
4,North East,Jharkhand,Rural,Ranchi,Personal,Entertainment,TV,0,0,0,0,1,2,25000.0,15,6,Dealer,1.3,Claim,0


In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 20 columns):
Region              358 non-null object
State               358 non-null object
Area                358 non-null object
City                358 non-null object
Consumer_profile    358 non-null object
Product_category    358 non-null object
Product_type        358 non-null object
AC_1001_Issue       358 non-null int64
AC_1002_Issue       358 non-null int64
AC_1003_Issue       358 non-null int64
TV_2001_Issue       358 non-null int64
TV_2002_Issue       358 non-null int64
TV_2003_Issue       358 non-null int64
Claim_Value         358 non-null float64
Service_Centre      358 non-null int64
Product_Age         358 non-null int64
Purchased_from      358 non-null object
Call_details        358 non-null float64
Purpose             358 non-null object
Fraud               358 non-null int64
dtypes: float64(2), int64(9), object(9)
memory usage: 56.1+ KB
None


As per the problem the issues and the service centre code are categorical variables

In [9]:
df = pd.DataFrame.copy(data)
columns = ['Product_Age','Call_details','Claim_Value']
for col in df.columns:
    if col not in columns:
        df[col] = df[col].astype('category')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 20 columns):
Region              358 non-null category
State               358 non-null category
Area                358 non-null category
City                358 non-null category
Consumer_profile    358 non-null category
Product_category    358 non-null category
Product_type        358 non-null category
AC_1001_Issue       358 non-null category
AC_1002_Issue       358 non-null category
AC_1003_Issue       358 non-null category
TV_2001_Issue       358 non-null category
TV_2002_Issue       358 non-null category
TV_2003_Issue       358 non-null category
Claim_Value         358 non-null float64
Service_Centre      358 non-null category
Product_Age         358 non-null int64
Purchased_from      358 non-null category
Call_details        358 non-null float64
Purpose             358 non-null category
Fraud               358 non-null category
dtypes: category(17), float64(2), int64(1)
memory usage: 18.

In [11]:
df_label = pd.DataFrame.copy(df)
for i in df_label.columns:
    if df_label[i].dtype.name =='category':
        encode = LabelEncoder()
        encode.fit(list(set(df_label[i])))
        df_label[i] = encode.transform(df_label[i])

In [12]:
df_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358 entries, 0 to 357
Data columns (total 20 columns):
Region              358 non-null int32
State               358 non-null int32
Area                358 non-null int32
City                358 non-null int32
Consumer_profile    358 non-null int32
Product_category    358 non-null int32
Product_type        358 non-null int32
AC_1001_Issue       358 non-null int64
AC_1002_Issue       358 non-null int64
AC_1003_Issue       358 non-null int64
TV_2001_Issue       358 non-null int64
TV_2002_Issue       358 non-null int64
TV_2003_Issue       358 non-null int64
Claim_Value         358 non-null float64
Service_Centre      358 non-null int64
Product_Age         358 non-null int64
Purchased_from      358 non-null int32
Call_details        358 non-null float64
Purpose             358 non-null int32
Fraud               358 non-null int64
dtypes: float64(2), int32(9), int64(9)
memory usage: 43.5 KB


### Normalisation

In [14]:
df_norm = pd.DataFrame.copy(df)
column_norm = []
for i in df_norm.columns:
    if df_norm[i].dtype.name == 'category':
        column_norm.append(i)
df_dummy = pd.get_dummies(df_norm, columns =column_norm[:-1])
min_max_scaler = MinMaxScaler()
Scaled = min_max_scaler.fit_transform(df_dummy[['Product_Age','Call_details','Claim_Value']] )

In [16]:
col = list(df_dummy.columns)
norm_dummy = np.concatenate((Scaled,df_dummy.values[:,3:99]),axis=1)
df_norm_dummy = pd.DataFrame(norm_dummy,columns=col)
df_norm_dummy

,Claim_Value,Product_Age,Call_details,Fraud,Region_East,Region_North,Region_North East,Region_North West,Region_South,Region_South East,...,Service_Centre_13,Service_Centre_14,Service_Centre_15,Service_Centre_16,Purchased_from_Dealer,Purchased_from_Internet,Purchased_from_Manufacturer,Purpose_Claim,Purpose_Complaint,Purpose_Other
0,0.0576923,0,0.3,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
1,0.00708502,0.0169492,0.4,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,0.00708502,0.0305085,0.36,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
3,0.0172065,0.0508475,0.24,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,1,0
4,0.00303644,0.0271186,0.5,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
354,0.00910931,0.0372881,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
355,0.254049,0.288136,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
356,0.00910931,0.0338983,1,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0


In [17]:
sm_x_lab = pd.DataFrame.copy(df_label)
sm_y_lab = sm_x_lab.pop('Fraud')
sm = SMOTE(random_state = 1)
X_lab_balance, Y_lab_balance = sm.fit_sample(sm_x_lab, sm_y_lab)

In [19]:
df_majority = df[df.Fraud==0]
df_minority = df[df.Fraud==1]
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=323,    # to match majority class
                                 random_state=101) # reproducible results
df_balance = pd.concat([df_majority, df_minority_upsampled])
df_balance.Fraud.value_counts()
X_cat_balance = pd.DataFrame.copy(df_balance)
Y_cat_balance = X_cat_balance.pop('Fraud')
X_cat_balance.shape, Y_cat_balance.shape

((646, 19), (646,))

In [20]:
sm_x_nd = pd.DataFrame.copy(df_norm_dummy)
sm_y_nd = sm_x_nd.pop('Fraud')
sm = SMOTE(random_state =101)
X_nd_balance, Y_nd_balance = sm.fit_sample(sm_x_nd, sm_y_lab)
X_nd_balance.shape, Y_nd_balance.shape

((646, 97), (646,))

In [21]:
# Train test split for balanced data, for non tree based models
X_train_cat_bal,X_test_cat_bal,Y_train_cat_bal,Y_test_cat_bal = train_test_split(X_cat_balance, Y_cat_balance,test_size=0.3,random_state=101)
X_train_cat_bal.shape,X_test_cat_bal.shape,Y_train_cat_bal.shape,Y_test_cat_bal.shape

((452, 19), (194, 19), (452,), (194,))

In [22]:
# Train test split for original data, for tree based models
X_c = pd.DataFrame.copy(df)
Y_c = X_c.pop('Fraud')
X_train_cat,X_test_cat,Y_train_cat,Y_test_cat = train_test_split(X_c,Y_c,test_size=0.3,random_state=101)
X_train_cat.shape,X_test_cat.shape,Y_train_cat.shape,Y_test_cat.shape

((250, 19), (108, 19), (250,), (108,))

In [23]:
X_train_lab_bal,X_test_lab_bal,Y_train_lab_bal,Y_test_lab_bal = train_test_split(X_lab_balance, Y_lab_balance,test_size=0.3,random_state=101)
X_train_lab_bal.shape,X_test_lab_bal.shape,Y_train_lab_bal.shape,Y_test_lab_bal.shape

((452, 19), (194, 19), (452,), (194,))

In [25]:
# Original
X_l = pd.DataFrame.copy(df_label)
Y_l = X_l.pop('Fraud')
X_train_lab,X_test_lab,Y_train_lab,Y_test_lab = train_test_split(X_l,Y_l,test_size=0.3,random_state=101)
X_train_lab.shape,X_test_lab.shape,Y_train_lab.shape,Y_test_lab.shape

((250, 19), (108, 19), (250,), (108,))

In [26]:
# Balanced
X_train_nd_bal,X_test_nd_bal,Y_train_nd_bal,Y_test_nd_bal = train_test_split(X_nd_balance, Y_nd_balance,test_size=0.3,random_state=101)
X_train_nd_bal.shape,X_test_nd_bal.shape,Y_train_nd_bal.shape,Y_test_nd_bal.shape

((452, 97), (194, 97), (452,), (194,))

In [27]:
X_d = pd.DataFrame.copy(df_norm_dummy)
Y_d = X_d.pop('Fraud')
X_train_nd,X_test_nd,Y_train_nd,Y_test_nd = train_test_split(X_d,Y_d,test_size=0.3,random_state=101)
X_train_nd.shape,X_test_nd.shape,Y_train_nd.shape,Y_test_nd.shape

((250, 97), (108, 97), (250,), (108,))

### Decision Tree

In [28]:
def treebuild(cri='entropy',mxd=10,minsl=2,rs=28,spl='best'):
    Tree = DecisionTreeClassifier(criterion=cri,max_depth=mxd,min_samples_leaf=minsl,random_state=rs,splitter=spl)
    Tree.fit(X_train_lab_bal,Y_train_lab_bal)
    pred_bal = Tree.predict(X_test_lab_bal)
    pred_ = Tree.predict(X_test_lab)
    prr_bal = Tree.predict(X_train_lab_bal)
    prr_ = Tree.predict(X_train_lab)
    print("Test Accuracy original Data",np.mean(Y_test_lab==pred_))
    print("Train Accuracy Original Data",np.mean(Y_train_lab==prr_))
    print ("Test Accuracy Balanced Data",np.mean(Y_test_lab_bal==pred_bal))
#     print(classification_report(Y_test_lab_bal,pred_bal))
    print("Train Accuracy Balanced Data",np.mean(Y_train_lab_bal==prr_bal))
    print(classification_report(Y_test_lab,pred_))
interact(treebuild,cri=['entropy','gini'],mxd=[i for i in range(1,20)],minsl=[i for i in range(1,10)],rs=[i for i in  range(30)],spl=['best','random'])

interactive(children=(Dropdown(description='cri', options=('entropy', 'gini'), value='entropy'), Dropdown(desc…

<function __main__.treebuild(cri='entropy', mxd=10, minsl=2, rs=28, spl='best')>

### Random forest

In [29]:
def forests(n_est=20,cri='entropy',mxd=9,mslf=3,mf='auto',rs=9):
    forest = RandomForestClassifier(n_estimators=n_est,criterion=cri,max_depth=mxd,min_samples_leaf=mslf,max_features=mf,random_state=rs)
    forest.fit(X_train_lab_bal,Y_train_lab_bal)
    pred_bal = forest.predict(X_test_lab_bal)
    pred_ = forest.predict(X_test_lab)
    prr_bal = forest.predict(X_train_lab_bal)
    prr_ = forest.predict(X_train_lab)
    print("Test Accuracy original Data",np.mean(Y_test_lab==pred_))
    print(classification_report(Y_test_lab,pred_))
    print ("Test Accuracy Balanced Data",np.mean(Y_test_lab_bal==pred_bal))
    print(classification_report(Y_test_lab_bal,pred_bal))
    print("Train Accuracy Original Data",np.mean(Y_train_lab==prr_))
    print("Train Accuracy Balanced Data",np.mean(Y_train_lab_bal==prr_bal))
interact(forests,n_est=[i for i in range(10,100)],cri=['gini','entropy'],mxd=[i for i in range(1,10)],mslf=[i for i in range(1,10)],mf=['auto','sqrt','log2'],rs=[i for i in range(30)])

interactive(children=(Dropdown(description='n_est', index=10, options=(10, 11, 12, 13, 14, 15, 16, 17, 18, 19,…

<function __main__.forests(n_est=20, cri='entropy', mxd=9, mslf=3, mf='auto', rs=9)>

### XG Boost

In [30]:
xgm = XGBClassifier(max_depth=5,learning_rate=0.2,n_estimators=200)
xgm.fit(X_train_lab_bal,Y_train_lab_bal)
pred_bal = xgm.predict(X_test_lab_bal)
pred_ = xgm.predict(X_test_lab.values)
prr_bal = xgm.predict(X_train_lab_bal)
prr_ = xgm.predict(X_train_lab.values)

print("Train Accuracy Original Data",np.mean(Y_train_lab==prr_))
print("Test Accuracy original Data",np.mean(Y_test_lab==pred_))
print(classification_report(Y_test_lab,pred_))
print ("Test Accuracy Balanced Data",np.mean(Y_test_lab_bal==pred_bal))
# print(classification_report(Y_test_lab_bal,pred_bal))

print("Train Accuracy Balanced Data",np.mean(Y_train_lab_bal==prr_bal))

Train Accuracy Original Data 0.932
Test Accuracy original Data 0.9351851851851852
              precision    recall  f1-score   support

           0       0.96      0.97      0.96        96
           1       0.73      0.67      0.70        12

    accuracy                           0.94       108
   macro avg       0.84      0.82      0.83       108
weighted avg       0.93      0.94      0.93       108

Test Accuracy Balanced Data 0.8969072164948454
Train Accuracy Balanced Data 0.9911504424778761


### CAT Boost

In [31]:
modelcat = CatBoostClassifier(learning_rate=0.1,depth=3,n_estimators=400,cat_features=[0,1,2,3,4,5,6,7,8,9,10,11,12,14,16,18])
modelcat.fit(X_train_cat_bal,Y_train_cat_bal)
pred_bal = modelcat.predict(X_test_cat_bal)
pred_ = modelcat.predict(X_test_cat)
prr_bal = modelcat.predict(X_train_cat_bal)
prr_ = modelcat.predict(X_train_cat)

0:	learn: 0.6576987	total: 60.2ms	remaining: 24s
1:	learn: 0.6234631	total: 67ms	remaining: 13.3s
2:	learn: 0.6007976	total: 75.3ms	remaining: 9.96s
3:	learn: 0.5760930	total: 81.5ms	remaining: 8.06s
4:	learn: 0.5565750	total: 87.4ms	remaining: 6.9s
5:	learn: 0.5327687	total: 91.6ms	remaining: 6.01s
6:	learn: 0.5160054	total: 95.5ms	remaining: 5.36s
7:	learn: 0.4918527	total: 103ms	remaining: 5.06s
8:	learn: 0.4710912	total: 109ms	remaining: 4.74s
9:	learn: 0.4565568	total: 111ms	remaining: 4.32s
10:	learn: 0.4440749	total: 117ms	remaining: 4.14s
11:	learn: 0.4328959	total: 123ms	remaining: 3.97s
12:	learn: 0.4020611	total: 127ms	remaining: 3.77s
13:	learn: 0.3846938	total: 133ms	remaining: 3.66s
14:	learn: 0.3808300	total: 136ms	remaining: 3.5s
15:	learn: 0.3786545	total: 142ms	remaining: 3.41s
16:	learn: 0.3635161	total: 148ms	remaining: 3.34s
17:	learn: 0.3581137	total: 154ms	remaining: 3.28s
18:	learn: 0.3478843	total: 160ms	remaining: 3.21s
19:	learn: 0.3477591	total: 161ms	remain

160:	learn: 0.1457597	total: 966ms	remaining: 1.43s
161:	learn: 0.1453336	total: 972ms	remaining: 1.43s
162:	learn: 0.1449856	total: 978ms	remaining: 1.42s
163:	learn: 0.1447780	total: 986ms	remaining: 1.42s
164:	learn: 0.1444458	total: 992ms	remaining: 1.41s
165:	learn: 0.1443919	total: 998ms	remaining: 1.41s
166:	learn: 0.1436724	total: 1s	remaining: 1.4s
167:	learn: 0.1435988	total: 1.01s	remaining: 1.4s
168:	learn: 0.1434178	total: 1.02s	remaining: 1.39s
169:	learn: 0.1430715	total: 1.02s	remaining: 1.38s
170:	learn: 0.1429893	total: 1.03s	remaining: 1.38s
171:	learn: 0.1429447	total: 1.03s	remaining: 1.37s
172:	learn: 0.1416389	total: 1.04s	remaining: 1.36s
173:	learn: 0.1413784	total: 1.05s	remaining: 1.36s
174:	learn: 0.1404971	total: 1.05s	remaining: 1.35s
175:	learn: 0.1403283	total: 1.06s	remaining: 1.35s
176:	learn: 0.1396785	total: 1.06s	remaining: 1.34s
177:	learn: 0.1396105	total: 1.07s	remaining: 1.33s
178:	learn: 0.1392517	total: 1.08s	remaining: 1.33s
179:	learn: 0.138

339:	learn: 0.0949901	total: 2.04s	remaining: 360ms
340:	learn: 0.0946259	total: 2.05s	remaining: 355ms
341:	learn: 0.0946188	total: 2.06s	remaining: 349ms
342:	learn: 0.0945198	total: 2.06s	remaining: 342ms
343:	learn: 0.0943366	total: 2.06s	remaining: 336ms
344:	learn: 0.0943226	total: 2.07s	remaining: 330ms
345:	learn: 0.0943141	total: 2.08s	remaining: 324ms
346:	learn: 0.0941521	total: 2.08s	remaining: 318ms
347:	learn: 0.0940997	total: 2.09s	remaining: 312ms
348:	learn: 0.0937064	total: 2.1s	remaining: 306ms
349:	learn: 0.0935867	total: 2.1s	remaining: 300ms
350:	learn: 0.0935065	total: 2.11s	remaining: 294ms
351:	learn: 0.0934353	total: 2.11s	remaining: 288ms
352:	learn: 0.0934241	total: 2.12s	remaining: 282ms
353:	learn: 0.0930615	total: 2.13s	remaining: 276ms
354:	learn: 0.0930370	total: 2.13s	remaining: 270ms
355:	learn: 0.0929055	total: 2.14s	remaining: 264ms
356:	learn: 0.0926892	total: 2.14s	remaining: 258ms
357:	learn: 0.0921845	total: 2.15s	remaining: 252ms
358:	learn: 0.

In [32]:
print("Test Accuracy original Data",np.mean(Y_test_cat==pred_))
print(classification_report(Y_test_cat,pred_))
print ("Test Accuracy Balanced Data",np.mean(Y_test_cat_bal==pred_bal))
# print(classification_report(Y_test_cat_bal,pred_bal))
print("Train Accuracy Original Data",np.mean(Y_train_cat==prr_))
print("Train Accuracy Balanced Data",np.mean(Y_train_cat_bal==prr_bal))

Test Accuracy original Data 0.9444444444444444
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        96
           1       0.67      1.00      0.80        12

    accuracy                           0.94       108
   macro avg       0.83      0.97      0.88       108
weighted avg       0.96      0.94      0.95       108

Test Accuracy Balanced Data 0.9381443298969072
Train Accuracy Original Data 0.916
Train Accuracy Balanced Data 0.9668141592920354


In [33]:
Proba = pd.DataFrame(modelcat.predict_proba(X_test_cat))
Proba.columns = ['Fraud_no','Fraud_yes']
Proba['Actual'] = list(Y_test_cat)
Proba['Predicted'] = list(pred_)
Proba.loc[(Proba.Actual!=Proba.Predicted)]

,Fraud_no,Fraud_yes,Actual,Predicted
4,0.130851,0.869149,0,1
24,0.125887,0.874113,0,1
28,0.060295,0.939705,0,1
31,0.101954,0.898046,0,1
53,0.228867,0.771133,0,1
61,0.260795,0.739205,0,1


### Logistic Regression

In [34]:
methods = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
def Logitt(method):
    lr1 = LogisticRegression(solver=method) 
    lr1.fit(X_train_nd_bal,Y_train_nd_bal)
    pred_bal = lr1.predict(X_test_nd_bal)
    pred_ = lr1.predict(X_test_nd)
    prr_bal = lr1.predict(X_train_nd_bal)
    prr_ = lr1.predict(X_train_nd)
    print("Test Accuracy original Data",np.mean(Y_test_nd==pred_))
    print(classification_report(list(Y_test_nd),pred_))
    print ("Test Accuracy Balanced Data",np.mean(list(Y_test_nd_bal)==pred_bal))
#     print(classification_report(list(Y_test_nd_bal),pred_bal))
    print("Train Accuracy Original Data",np.mean(list(Y_train_nd)==prr_))
    print("Train Accuracy Balanced Data",np.mean(list(Y_train_nd_bal)==prr_bal))
interact(Logitt,method = methods)

interactive(children=(Dropdown(description='method', options=('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'…

<function __main__.Logitt(method)>

### Support Vector Machine

In [35]:
svclassifier = SVC(kernel = 'linear')
svclassifier.fit(X_train_nd_bal, Y_train_nd_bal)
pred_bal = svclassifier.predict(X_test_nd_bal)
pred_ = svclassifier.predict(X_test_nd)
prr_bal = svclassifier.predict(X_train_nd_bal)
prr_ = svclassifier.predict(X_train_nd)
print("Test Accuracy original Data",np.mean(Y_test_nd==pred_))
print(classification_report(list(Y_test_nd),pred_))
print ("Test Accuracy Balanced Data",np.mean(list(Y_test_nd_bal)==pred_bal))
# print(classification_report(list(Y_test_nd_bal),pred_bal))
print("Train Accuracy Original Data",np.mean(list(Y_train_nd)==prr_))
print("Train Accuracy Balanced Data",np.mean(list(Y_train_nd_bal)==prr_bal))

Test Accuracy original Data 0.8240740740740741
              precision    recall  f1-score   support

           0       0.99      0.81      0.89        96
           1       0.38      0.92      0.54        12

    accuracy                           0.82       108
   macro avg       0.68      0.86      0.71       108
weighted avg       0.92      0.82      0.85       108

Test Accuracy Balanced Data 0.8556701030927835
Train Accuracy Original Data 0.828
Train Accuracy Balanced Data 0.9092920353982301
